In [1]:
from importlib import reload
import sys
import os
sys.path.insert(0, './../scripts')

In [2]:
import pickle
import numpy as np
import matplotlib as mpl
from matplotlib import cm
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.figure import figaspect
import time

import spat_ring_network as r_network
import spat_snp_network as m_network
import sim_util as su
import ricciardi as ric
import integrate as integ

def imshowbar(ax,A,**kwargs):
    imsh = ax.imshow(A,**kwargs)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes('right', size='5%', pad=0.05)
    fig.colorbar(imsh, cax=cax, orientation='vertical')

# Read Results Files

In [55]:
max_njob = 1

res_dict = {}

nsim = 0
for njob in range(max_njob):
    try:
        with open('./../results/results_base_'+str(njob)+'.pkl', 'rb') as handle:
            this_res_dict = pickle.load(handle)
        max_rep = max(list(this_res_dict.keys())) + 1
        for idx_rep in range(max_rep):
            res_dict[nsim] = this_res_dict[idx_rep]
            nsim += 1
    except:
        pass

print(nsim)

1


In [18]:
data_means = np.array([6.22, 6.72, 7.17, 7.67, 8.,  10.97, 16.7])
data_stds =  np.array([5.79, 6.64, 6.93, 7.15, 7.07, 8.98, 13.6])

data_means_err = np.array([0.83, 0.96, 1.00, 1.03, 1.02, 1.30, 2.00])
data_stds_err =  np.array([0.48, 0.78, 0.96, 0.81, 0.78, 1.08, 2.63])

In [49]:
from scipy.interpolate import RegularGridInterpolator
from scipy.optimize import least_squares

aXs = np.arange(1,15+2,4)
bXs = np.arange(1,7+2,4)
eXs = np.arange(0,0.3+0.05,0.1)

def fit_best_inputs(eX):
    def residuals(x):
        this_bX = x[0]
        this_aXs = np.concatenate(([0],x[1:]))
        pred_means = mean_itp(np.vstack((this_aXs,this_bX*np.ones(7),eX*np.ones(7))).T)
        pred_stds = std_itp(np.vstack((this_aXs,this_bX*np.ones(7),eX*np.ones(7))).T)
        res = np.array([(pred_means-data_means)/data_means_err, (pred_stds-data_stds)/data_stds_err])
        return res.ravel()
    xmin = np.concatenate(([bXs[ 0]],aXs[ 0]*np.ones(6)))
    xmax = np.concatenate(([bXs[-1]],aXs[-1]*np.ones(6)))
    x0 = np.concatenate(([2],np.linspace(1,12,6)))
    results = least_squares(residuals,x0,bounds=(xmin,xmax))
    this_bX = results.x[0]
    this_aXs = np.concatenate(([0],results.x[1:]))
    return (this_bX,this_aXs,mean_itp(np.vstack((this_aXs,this_bX*np.ones(7),eX*np.ones(7))).T),
            std_itp(np.vstack((this_aXs,this_bX*np.ones(7),eX*np.ones(7))).T),results.cost)

def fit_best_input_var():
    def residuals(x):
        _,_,_,_,cost = fit_best_inputs(x[0])
        return [cost]
    xmin,xmax = (eXs[0]),(eXs[-1])
    x0 = np.array([0.2])
    results = least_squares(residuals,x0,bounds=(xmin,xmax))
    return (results.x[0],*fit_best_inputs(results.x[0]))

In [45]:
means = res_dict[0]['means']
stds = res_dict[0]['stds']

mean_itp = RegularGridInterpolator((aXs,bXs,eXs), means, bounds_error=False, fill_value=None)
std_itp = RegularGridInterpolator((aXs,bXs,eXs), stds, bounds_error=False, fill_value=None)

In [50]:
best_eX,best_bX,best_aXs,best_means,best_stds,best_cost = fit_best_input_var()
print(best_eX)
print(best_bX)
print(best_aXs)
print(best_means)
print(best_stds)
print(best_cost)

0.29999999999998894
2.585610377954368
[ 0.          2.21622578  2.34301577  3.11526436  3.21549434  5.75817576
 10.60483713]
[ 7.54640802  8.9527195   9.0331744   9.52320663  9.58680781 11.31154654
 15.21630063]
[ 3.66307448  5.52929716  5.63606352  6.28635278  6.37075369  8.83473955
 15.17699909]
21.75416497356588


In [ ]:
input_fit_dict = {}

for idx_sim in range(nsim):
    this_res_dict = res_dict[idx_sim]

In [40]:
np.arange(1,9+2,4)

array([1, 5, 9])